### Determinando un FVG
Lo primero que debemos hacer es leer data.
Vamos a descargar data de BTCUSDT en el rango 2023-08-09 - 2023-08-11 (9 al 11 de Agosto de 2023) en la temporalidad M5

In [1]:
import pandas as pd
from lightweight_charts import Chart
import zipfile
import os

Este es un zip que tiene los datos de un solo dia 11 de Agosto en M5

In [2]:
zipfile_name = 'BTCUSDT-5m-2023-08-11.zip'
file_name = 'BTCUSDT-5m-2023-08-11.csv'
data_path = os.path.join('btc_fvg', zipfile_name)
# btc_fvg\BTCUSDT-5m-2023-08-11.zip

Leer el dataframe

In [3]:
def read_csv(data_path, file_name):
    """
    Function to read csv inside zip.
    Input path strings
    Output: Pandas Dataframe
    """ 
    with zipfile.ZipFile(data_path, 'r') as zip_ref:
        with zip_ref.open(file_name) as csv_file:
            # Lee el archivo CSV utilizando Pandas
            dataframe = pd.read_csv(csv_file)
            return dataframe


In [4]:
data = read_csv(data_path, file_name)
data.head()

,open_time,open,high,low,close,volume,close_time,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,ignore
0,1691712000000,29443.7,29443.8,29439.5,29439.5,142.335,1691712299999,4.190503e+06,1814,33.670,9.912864e+05,0
1,1691712300000,29439.6,29442.9,29436.0,29442.8,222.348,1691712599999,6.545603e+06,1957,149.725,4.407645e+06,0
2,1691712600000,29442.8,29450.3,29442.8,29449.0,314.360,1691712899999,9.257319e+06,2312,210.901,6.210576e+06,0
3,1691712900000,29449.0,29459.9,29445.5,29459.8,400.268,1691713199999,1.178984e+07,3253,244.879,7.212872e+06,0
4,1691713200000,29459.9,29475.0,29459.9,29475.0,397.210,1691713499999,1.170583e+07,3365,261.123,7.695420e+06,0


In [5]:
# Function to clean data to what i need

def prep_data(dataframe):
    # Eliminating irrelevant columns
    dataframe = dataframe[['open_time', 'open', 'high', 'low','close', 'volume']]
    dataframe['open_time'] = pd.to_datetime(dataframe['open_time'],unit='ms')
    return dataframe

In [6]:
data_cleaned = prep_data(data)

C:\Users\egrados\AppData\Local\Temp\ipykernel_25280\3479782272.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['open_time'] = pd.to_datetime(dataframe['open_time'],unit='ms')


In [7]:
# Rename dataframe to match lightweight charts requirement
data_cleaned = data_cleaned.rename(columns={"open_time": "time"})

# Setear el index como el timestamp de la
data_cleaned.set_index('time', inplace=True)

data_cleaned

,open,high,low,close,volume
time,,,,,
2023-08-11 00:00:00,29443.7,29443.8,29439.5,29439.5,142.335
2023-08-11 00:05:00,29439.6,29442.9,29436.0,29442.8,222.348
2023-08-11 00:10:00,29442.8,29450.3,29442.8,29449.0,314.360
2023-08-11 00:15:00,29449.0,29459.9,29445.5,29459.8,400.268
2023-08-11 00:20:00,29459.9,29475.0,29459.9,29475.0,397.210
...,...,...,...,...,...
2023-08-11 23:35:00,29416.1,29416.9,29413.4,29413.4,176.268
2023-08-11 23:40:00,29413.5,29420.0,29413.4,29419.9,139.629
2023-08-11 23:45:00,29420.0,29430.0,29412.2,29412.2,311.631


### Select date ranges from dataframe with timestamp index
Select an specific date range from the entire dataset, asumming the dataframe is clean and have *time,open,high,low,close,volume columns*

In [16]:
# Select test start and finish
start_date_1 = '2023-08-11 00:00:00'
finish_date_1 = '2023-08-11 01:00:00'

# Apply mask on function
def select_daterange(dataframe, start_date, finish_date):
    selected = dataframe.loc[start_date:finish_date]
    return selected

data_selected = select_daterange(data_cleaned,start_date_1,finish_date_1)
data_selected


,open,high,low,close,volume
time,,,,,
2023-08-11 00:00:00,29443.7,29443.8,29439.5,29439.5,142.335
2023-08-11 00:05:00,29439.6,29442.9,29436.0,29442.8,222.348
2023-08-11 00:10:00,29442.8,29450.3,29442.8,29449.0,314.360
2023-08-11 00:15:00,29449.0,29459.9,29445.5,29459.8,400.268
2023-08-11 00:20:00,29459.9,29475.0,29459.9,29475.0,397.210
2023-08-11 00:25:00,29475.0,29477.4,29468.7,29474.4,357.485
2023-08-11 00:30:00,29474.4,29474.5,29464.3,29466.0,245.952
2023-08-11 00:35:00,29466.0,29467.1,29464.2,29467.0,141.601
2023-08-11 00:40:00,29467.0,29489.5,29467.0,29489.4,571.195


Function to create our desired time ranges for start, which is 10am-11am NY time, traslated to UTC is 14:00-15:00.
It should iterate over all the days and select:
- Candles inside this timeframe.
- 1 candle prior to this timeframe, in order to be able to capture the first FVG.

In [43]:

for date in data_cleaned.index:
    if date 
    print (date)

2023-08-11 00:00:00
2023-08-11 00:05:00
2023-08-11 00:10:00
2023-08-11 00:15:00
2023-08-11 00:20:00
2023-08-11 00:25:00
2023-08-11 00:30:00
2023-08-11 00:35:00
2023-08-11 00:40:00
2023-08-11 00:45:00
2023-08-11 00:50:00
2023-08-11 00:55:00
2023-08-11 01:00:00


In [45]:
import pandas as pd

# Define el rango de tiempo deseado
start_time = pd.Timestamp('2023-08-11 10:00:00')
end_time = pd.Timestamp('2023-08-11 11:00:00')

# Itera sobre los índices del DataFrame
for timestamp in data_cleaned.index:
    # Formatea el índice de timestamp en formato de hora
    time_str = timestamp.strftime('%H:%M:%S')
    
    # Compara el tiempo formateado con el rango deseado
    if start_time <= timestamp <= end_time:
        print(timestamp)

2023-08-11 10:00:00
2023-08-11 10:05:00
2023-08-11 10:10:00
2023-08-11 10:15:00
2023-08-11 10:20:00
2023-08-11 10:25:00
2023-08-11 10:30:00
2023-08-11 10:35:00
2023-08-11 10:40:00
2023-08-11 10:45:00
2023-08-11 10:50:00
2023-08-11 10:55:00
2023-08-11 11:00:00


In [22]:
data_selected.index[1] - pd.Timedelta(minutes=10)

Timestamp('2023-08-10 23:55:00')

### FVG Logic

##### Bullish FVG
    Price.Low (last) is greater than Price.High (2 candles ago) – This defines the lack of overlap between candle wicks
    Price.Close (1 candle ago) is greater than Price.Open (1 candle ago) by at least ‘x %’ – This defines the large buying pressure
    Price.Close (1 candle ago) is greater than Price.High  (2 candles ago) – This defines that the candle with the buying pressure closed above the high of the previous candle

- Definimos un gap minimo, porcentual. GAP = %
- EL LOW de la vela actual (last) es *mayor* que el HIGH de la vela 2 posiciones anterior.
- El CLOSE de la vela anterior (1 posicion anterior) es *mayor* que el OPEN de la vela 1 posicion anterior por al menos el GAP porcentual.
- El CLOSE de la vela 1 posicion anterior es *mayor* que el HIGH de la vela 2 posisciones anterior.


##### Bearish FVG
    Price.High (last) is less than Price.Low (2 candles ago) – This defines the lack of overlap between candle wicks
    Price.Close (1 candle ago) is less than Price.Open (1 candle ago) by at least ‘x %’ – This defines the large selling pressure
    Price.Close (1 candle ago) is less than Price.Low (2 candles ago) – This defines that the candle with the selling pressure closed below the low of the previous candle

- Definimos un gap minimo, porcentual. GAP = %
- El HIGH de la vela actual (last) es *menor* que el LOW de la vela 2 posiciones anterior.
- El CLOSE de la vela 1 posicion anterior es *menor* que el OPEN de la vela 1 posicion anterior por al menos el GAP porcentual.
- El CLOSE de la vela 1 posicion anterior es *menor* que el LOW de la vela 2 posiciones anterior.

In [9]:
# funcion para determinar FVG. SE dejará para después.

def fvg_detector(dataframe):
    # Define porcentual GAP
    percentual_gap = 1.00003
    
    # Obtain df lenght
    lenght = len(dataframe)
    
    # Entramos por cada fila
    for candle in range(lenght):
        #print(f"La vela {candle} tiene como open price {dataframe.iloc[candle]['open']}")
        # Define gap in USD:
        usd_gap = dataframe.iloc[candle-2]['high']*percentual_gap

        if candle >= 2:
            # Bulish FVG
            if dataframe.iloc[candle]['low'] > dataframe.iloc[candle-2]['high']\
            and dataframe.iloc[candle-1]['close'] - dataframe.iloc[candle-1]['open']\
            and dataframe.iloc[candle-1]['close'] > dataframe.iloc[candle-2]['high']:
            #and dataframe.iloc[candle]['close'] > dataframe.iloc[candle]['open']
                print(f"La vela {candle} generó un Bullish FVG en la vela {candle-1}, lowbarrier: {dataframe.iloc[candle-2]['high']}, high barrier: {dataframe.iloc[candle]['low']}")
            
            # Bearish FVG:
            if dataframe.iloc[candle]['high'] < dataframe.iloc[candle-2]['low']\
            and dataframe.iloc[candle-1]['close'] < dataframe.iloc[candle-1]['open']\
            and dataframe.iloc[candle-1]['close'] < dataframe.iloc[candle-2]['low']:
            #and dataframe.iloc[candle]['close'] < dataframe.iloc[candle]['open']
                print(f"La vela {candle} generó un Bearish FVG en la vela {candle-1}, lowbarrier: {dataframe.iloc[candle]['high']}, high barrier: {dataframe.iloc[candle-2]['low']}")

    return
    



fvg_detector(data_cleaned)

La vela 3 generó un Bullish FVG en la vela 2, lowbarrier: 29442.9, high barrier: 29445.5
La vela 4 generó un Bullish FVG en la vela 3, lowbarrier: 29450.3, high barrier: 29459.9
La vela 5 generó un Bullish FVG en la vela 4, lowbarrier: 29459.9, high barrier: 29468.7
La vela 7 generó un Bearish FVG en la vela 6, lowbarrier: 29467.1, high barrier: 29468.7
La vela 9 generó un Bullish FVG en la vela 8, lowbarrier: 29467.1, high barrier: 29470.0
La vela 13 generó un Bearish FVG en la vela 12, lowbarrier: 29466.2, high barrier: 29468.4
La vela 15 generó un Bullish FVG en la vela 14, lowbarrier: 29466.2, high barrier: 29469.5
La vela 17 generó un Bearish FVG en la vela 16, lowbarrier: 29459.7, high barrier: 29469.5
La vela 18 generó un Bearish FVG en la vela 17, lowbarrier: 29456.4, high barrier: 29458.2
La vela 19 generó un Bearish FVG en la vela 18, lowbarrier: 29444.9, high barrier: 29454.8
La vela 20 generó un Bearish FVG en la vela 19, lowbarrier: 29439.9, high barrier: 29444.8
La vela 2